***

* [Outline](../0_Introduction/0_introduction.ipynb)
* [Glossary](../0_Introduction/1_glossary.ipynb)
* [2. Mathematical Groundwork](2_0_introduction.ipynb)
    * Previous: [2.8 The Discrete Fourier Transform (DFT) and the Fast Fourier Transform (FFT)](2_8_the_discrete_fourier_transform.ipynb)
    * Next: [2.10 Linear Algrebra](2_10_linear_algebra.ipynb)

***

Import standard modules:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import HTML 
HTML('../style/course.css') #apply general CSS

Import section specific modules:

In [ ]:
from IPython.display import HTML
from ipywidgets import interact
HTML('../style/code_toggle.html')

## 2.9 Sampling Theory<a id='math:sec:sampling_theory'></a>

Our goal in radio interferometry is to produce a map of the sky by sampling a finite number of spatial frequency modes present in its spectrum (this will become clearer as the course unfolds). By sampling only a discrete number of frequency modes we are effectively turning a continuous problem into a discrete one. Recall that at the end of the last section we saw some peculiarities in the output of the discrete Fourier transform (DFT). In particular we note the following features:

   * That there were non-zero components $Y_k$ at frequencies which were not present in the input signal.
   * That the amplitudes of the $Y_k$ corresponding to the frequencies present in the input signal were not all equal dispite the fact that they are the same in the input signal.
   * That is was not possible to find the frequencies present in the input signal when $N$ (the number of samples) was below a certain number. 

These features will, in some form or another, also be present in radio interferometry. The aim of this section is to develop an intuitive understanding of why the above features are present in the discrete spectrum. In doing so we will discuss aliasing, the Shannon-Nyquist sampling theorem (or just sampling theorem for short), impulse response and deconvolution. A proper understanding of these topics will allow us to answer questions such as:

* Given the (effective) diameter of a telescope, what is the maximum pixel size we can choose for the reconstructed image?
* Which of the features in the reconstructed image are real features corresponding to the sky and which are artificial features introduced by sampling only a limited number of frequency modes?

Note that, in interferometry, we are sampling in the spatial frequency domain. Unfortunately, our intuition does not extend as easily to this domain. We are more inclined to think of the world in terms of a sequence of events i.e. in the time domain. A similar statement is true in interferometry. Our intuition extends more easily to the image domain than to its spectral dual. As a result we will start this section by considering sampling in the time domain and look at the implications it has for the frequency domain.

9. [Sampling theory](2_9_sampling_theory.ipynb) 
    1. [Sampling a continuous function](#math:sec:sampling_continuous_function)
    2. [Nyquist's sampling theorem](#math:sec:nyquists_sampling_theorem)
    3. [Aliasing](#math:sec:aliasing)
    4. [Limited sampling](#math:sec:limited_sampling)
    5. [Examples for aliasing and limited sampling](#math:sec:examples_for_aliasing_and_limited_sampling)

### 2.9.1 Sampling a Continuous Function <a id='math:sec:sampling_continuous_function'></a>

Intuitively we can think of the act of sampling simply as selecting a (possibly infinite) number of points of a continuous function. For simplicity let's start by considering a real valued function 

$$ f:\mathbb{R} \rightarrow \mathbb{R}. $$

The mathematical equivalent of selecting an infinite number of equally spaced samples can be expressed as 

$$ f(t_n) = \sum_{n = -\infty}^{\infty} f(t) \delta(t - n\Delta t) = f(t)\frac{1}{\Delta t}III(\frac{t}{\Delta t}), $$

where $\Delta t$ is the sampling interval, the index $n$ labels the samples and $III(\cdot)$ is the Dirac comb (Shah function). Obviously it is not possible to sample a function at an infinite number of points in practice. Suppose we sample a function at $N$ points in a finite domain, $t_0 \leq t \leq t_f$ say. This can be achieved as follows

$$ f(t_n) = \sum_{n = 0}^{N-1} f(t) \delta(t - n\Delta t - t_0), \quad \mbox{where} \quad \Delta t = \frac{t_f - t_0}{N-1}, \quad \mbox{and} \quad t_n = t_0 + n\Delta t. $$

So far we have done nothing fancy, we have just expressed the act of sampling in terms of a continuous mathematical function. We can go a step further and write it purely in terms of the special functions introduced in ([$\S$ 2.2](2_2_important_functions.ipynb#math:eq:1_003) <!--\ref{math:eq_1_003}-->) as

$$f(t_n)= f(t)\frac{1}{\Delta t}III(\frac{t}{\Delta t})\sqcap_{t_0,t_f}(t), $$

where $\sqcap_{a,b}(\cdot)$ is the boxcar function. The sampling function can therefore be expressed as

$$ s_{t_0,\Delta t, N} = \frac{1}{\Delta t}III(\frac{t}{\Delta t})\sqcap_{t_0,t_f}(t),   $$

where it should be understood that the function $s_{t_0,\Delta t, N} $ implies a sampling of $N$ points in the interval $t_0 \leq t \leq t_f$ where $t_f = (N-1)\Delta t$. 

At this stage you might be rolling your eyes at this seemingly cumbersome notation but it does serve a purpose. But, let's first investigate what happens when we take the Fourier transform of the sampled function $f(t_n)$. First, recall the definition of the 1-D Fourier transform

$$ \mathscr{F}\{f(t)\}(s) = \int_{-\infty}^{\infty} f(t)e^{-2\pi\imath t s} dt. $$

Substituting in the sampled function we get

$$ \mathscr{F}\{f(t_n)\}(s) = \int_{-\infty}^{\infty} f(t)\frac{1}{\Delta t}III(\frac{t}{\Delta t})\sqcap_{t_0,t_f}(t)e^{-2\pi\imath t s} dt. $$

Next the Dirac comb changes the continuous integral into a discrete sum i.e.

$$ \mathscr{F}\{f(t_n)\}(s) = \sum_{n = -\infty}^{\infty} f(t_n) \sqcap_{t_0,t_f}(t)e^{-2\pi\imath t_n s}. $$

The boxcar function sets at the terms for which $n \notin [0,\cdots,N-1]$ to zero. Thus we have

$$ \mathscr{F}\{f(t_n)\}(s) = \sum_{n = 0}^{N-1} f(t_n) e^{-2\pi\imath t_n s}.  $$

At this stage you can probably spot where we are going with this. Defining

$$ y_n = e^{-2\pi\imath t_0}f(t_n) \quad \mbox{where} \quad t_n = t_0 + n\Delta t, $$

we see that

$$ \mathscr{F}\{f(t_n)\}(s) = \sum_{n = 0}^{N-1} f(t_n) e^{-2\pi\imath t_n s} = \sum_{n = 0}^{N-1} y_n e^{-2\pi\imath n\Delta t s}. $$

Finally, defining 

$$ s_k = \frac{k}{\Delta t N}, $$

gives

$$ \mathscr{F}\{y_n\}(s)_k = Y_k = \sum_{n = 0}^{N-1} y_n e^{-2\pi\imath \frac{n k}{N}}. $$

Note that, if $t_0 = 0$, we recover the Discrete Fourier Transform (DFT) as the Fourier transform of a sampled function $f(t_n)$. The factor $e^{-2\pi\imath t_0}$ is just a phase shift, it does not alter the amplitude of the components $Y_k$ at all (for a fixed time interval). 

Writing the DFT as the Fourier transform of a sampled signal allows us to understanding some the peculiarities about the DFT we noted in the introduction. To see this, recall that multiplication in the time domain is the same as convolution in the frequency domain i.e.

$$ z(t) = f(t)g(t) \quad \Rightarrow \quad \mathscr{F}\{z\}(s) = \mathscr{F}\{f\}(s) \circ \mathscr{F}\{g\}(s). $$

Applying this result to our sampled function we see that

$$ \mathscr{F}\{f(t_n)\}(s) =  \mathscr{F}\{f(t)\frac{1}{\Delta t}III(\frac{t}{\Delta t})\} \circ  \mathscr{F}\{\sqcap_{t_0,t_f}(t)\}. $$

If we also use the fact that the Fourier transform of the boxcar function is given by (might be a good idea for you to verify this)

$$ \mathscr{F}\{\sqcap_{t_0,t_f}(t)\} = (N-1)\Delta t ~ sinc ((N-1)\Delta t s), $$

then we have established that

$$ \mathscr{F}\{f(t_n)\}(s) =  \mathscr{F}\{f(t)\frac{1}{\Delta t}III(\frac{t}{\Delta t})\} \circ  (N-1)\Delta t ~ sinc ((N-1)\Delta t s). $$

We see here that the output of the DFT is the convolution of a sinc function with the Fourier transform of an infinitely sampled function. This explains why there are non-zero components of $Y_k$ even at frequencies which do not correspond to any of the frequencies in the input signal. It is caused by the fact that, in practice, we can only compute the DFT of a limited number of samples. Before we discuss the concept of aliasing, use the interactive demo below to convince yourself (by adjusting the $N$ slider from the far left to the far right) that the width of the sinc function decreases with increased number of samples (for a fixed time interval). Also check that, for a fixed total time interval and as long as we have a sufficient number of sampls, the starting value $t_0$ only changes the phase of $Y_k$ but not its amplitude. Note we are only plotting the components of $Y_k$ corresponding to frequencies below $f_k \leq 5$Hz. 

In [ ]:
def inter_DFT(N,t0,tf,f1=1,f2=2,f3=3,plot_interval=5.0,plot_phase=True,show_Nyquist=False):
    """
    Interactive DFT visualizer
    """
    #set time domain
    t = np.linspace(t0,tf,N)
    #Get the signal
    y = np.sin(2.0*np.pi*f1*t) + np.sin(2.0*np.pi*f2*t) + np.sin(2.0*np.pi*f3*t)
    #Take the DFT (here we use FFT for speed)
    Y = np.fft.fft(y)
    #Get sampling interval
    delt = (tf - t0)/(N-1)
    #Sampling rate
    fs = 1.0/delt
    #Covert k to frequency
    k = np.arange(N)
    fk = k*fs/N    
    #Plot amplitude and phase
    plt.figure(figsize=(15, 6))
    plt.subplot(121)
    absY = abs(Y)
    Ymax = np.unique(absY.max())[0]
    plt.stem(fk,absY)
    plt.xlabel('$f_k$',fontsize=18)
    plt.ylabel(r'$|Y_k|$',fontsize=18)
    plt.xlim(0,plot_interval)
    #Compute Nyquist freq
    f_N = (N-1)/(2*(tf - t0))
    if show_Nyquist and f_N < plot_interval:
        plt.arrow(f_N, 0, 0, Ymax, head_width=0.5, head_length=3.0, fc='r', ec='k',label='Nyquist')
        plt.legend()
    plt.subplot(122)
    if plot_phase:
        plt.stem(fk,np.angle(Y))
        plt.xlabel('$f_k$',fontsize=18)
        plt.ylabel(r'phase$(Y_k)$',fontsize=18)
        plt.xlim(0,plot_interval)
        plt.show()
    else:
        #Here we plot the theoretical spectrum
        ft = np.array([f1,f2,f3])
        Ymax = absY.max()
        Yt = np.array([Ymax,Ymax,Ymax])
        plt.stem(ft,Yt,label='Theoretical')
        plt.xlabel('$f_k$',fontsize=18)
        plt.ylabel(r'|Y_k|$',fontsize=18)
        plt.xlim(0,plot_interval)
        plt.arrow(f_N, 0, 0, Ymax, head_width=0.5, head_length=3.0, fc='r', ec='k',label='Nyquist')
        plt.legend()
        

In [ ]:
interact(lambda N,t0:inter_DFT(N=N,t0=t0,tf=t0+10),
                N=(50,512,1),t0=(0,5,0.01)) and None

Can you explain what happens to the $|Y_k|$ as $N \rightarrow 0$? Why do the amplitudes change? 

### 2.9.2 Nyquist-Shannon Sampling Theorem<a id='math:sec:nyquists_sampling_theorem'></a>

Hopefully, you will have noticed that the information contained in $Y_k$ is very sensitive to the sampling interval $\Delta t$. The Nyquist-Shannon sampling theorem explains why it is not possible to recover the frequency information contained in a signal when the sampling becomes too course i.e. $\Delta t$ is too large or $N$ is too small. One way to state the theorem is that, when sampling a signal at an interval $\Delta t$ (equivalently a frequency of $f_s = \frac{1}{\Delta t}$), it is not possible to recover any frequency information above a frequency of $f_N = \frac{1}{2\Delta t}$. We call $f_N$ the *Nyquist frequency*. In the above example we kept the sampling interval fixed. Thus, by decreasing $N$, we increase the width of the sampling interval $\Delta t$ and hence decrease $f_N$. This explains the seemingly erratic behaviour of $|Y_k|$ as $N \rightarrow 0$. Note that, at $f_1 = 1$Hz, $f_2 = 2$Hz and $f_3 = 3$Hz, the input frequencies remain the same. Decreasing $N$ below a certain limit therefore results in a value of $f_N$ less than the maximum frequency present in the signal. Proving the sampling theorem is trivial once the Poisson summation formula has been established. Recall that the periodic summation of the spectrum $Y(f)$ of the (Schwartz) function $y(t)$ is related to the Fourier series components of $y_n = \Delta t y(n \Delta t)$ according to 

$$ Y_{f_s}(f) = \sum_{k = -\infty}^{\infty} Y(f - kf_s) = \sum_{n = -\infty}^{\infty} y_n e^{-2 \pi \imath f \Delta t n} = \sum_{n = -\infty}^{\infty} \Delta t ~ y(\Delta t n) e^{-2\pi\imath f \Delta t n}. $$

Since we can obtain the $y_n$ simply by sampling the function $y(t)$ (and multiplying by $\Delta t$), and we can in principle compute the Fourier series of $y_n$ up to arbitrary order, it is possible to find $Y_{f_s}(f)$ up to arbitrary precision. If there are no frequencies greater than $\frac{f_s}{2} = f_N$ present in the signal, the spectrum $Y(f)$ can be found from $Y_{f_s}(f)$ simply by constructing the filter

$$ Y(f) = \sqcap(\frac{f}{f_s}) Y_{f_s}(f), $$ 

where $\sqcap(\cdot)$ is the rectangle (or boxcar) function. Finally, since the spectrum of a function completely determines the function, we must conclude that the samples $y(n\Delta t)$ are sufficient to reconstruct $y(t)$ completely. To see how this can be done, we simply note the following

\begin{eqnarray}
Y(f) &=& \sqcap(\frac{f}{f_s}) Y_{f_s}(f), \\ 
&=& \sqcap(\frac{f}{f_s}) \sum_{n = -\infty}^{\infty} \Delta t ~ y(\Delta t n) e^{-2\pi\imath f \Delta t n},\\
&=& \sum_{n = -\infty}^{\infty} y(\Delta t n) \Delta t ~ \sqcap(\Delta t ~ f) e^{-2\pi\imath f \Delta t n},\\
&=& \sum_{n = -\infty}^{\infty} y(\Delta t n) \mathscr{F}\{sinc\left(\frac{t - n\Delta t}{\Delta t}\right)\},
\end{eqnarray}

where we have used the formula for the Fourier transform of the rectangle function. Taking the inverse Fourier transform of both sides results in the Whittaker-Shannon interpolation formula

$$ y(t) = \sum_{n =-\infty}^{\infty} y(n\Delta t) ~ sinc\left(\frac{t - n\Delta t}{\Delta t}\right). $$ 

Note that, in order to use this formula, we still need an infinite number of samples of $y(t)$, something which is impossible in practice. 

### 2.9.3 Aliasing <a id='math:sec:aliasing'></a>

We have already touched upon the concept of aliasing when we discussed periodic summation. We noted that the Discrete Time Fourier Transform (DTFT) results in a periodic function of the frequency variable $Y_{f_s}(f)$ which is in some way related to the periodic summation of the spectrum $Y(f)$ of the signal $y(t)$. The copies of $Y(f)$ in $Y_{f_s}(f)$ at $k \neq 0$ were called aliases. Understanding aliasing in the context of the DFT is slightly more subtle and is probably best illustrated with an example.  

Going back to our sum of sine functions model above, let's consider what happens if we now allow one of the frequency components to vary i.e. 

$$ y = \sin(2\pi f_1 t) + \sin(2\pi f_2 t) + \sin(2\pi f_i t), $$

where $f_i$ is the varaible frequency. If we sample this function in the interval $0< t < 10$s with 256 samples (i.e. $\Delta t = \frac{t_f - t_0}{N - 1} = \frac{10}{255} \approx 0.0392s$) we obtain a Nyquist frequency of $f_N = \frac{1}{2\Delta t} = 12.75$. 

In [ ]:
interact(lambda fi:inter_DFT(N=256,t0=0.0,tf=10,f3=fi,plot_interval=26,plot_phase=False,show_Nyquist=True),
                fi=(3,25,0.25)) and None

### 2.9.4 Demonstrations of Aliasing and the Sampling Theorem<a id='math:sec:examples_for_aliasing_and_limited_sampling'></a>

In [ ]:
f = 4 # frequency of the physical signal
plt.figure(figsize=(8, 6))
plt.axhline(y=0, xmin=0, xmax=250, color='r')
plt.plot(np.linspace(0,1,100.), np.sin(2*np.pi*f*np.linspace(0,1,100.)), 'k')
plt.show()

Now supose that this  sine wave is the physical signal i.e the continuous time variant signal. If we sample this signal with sampling intervals of  $\Delta x=1/80$, $\Delta x=1/50$ and $\Delta x=1/25$ then we have the following.

In [ ]:
y = np.sin(2*np.pi*f*np.linspace(0,1,80.))
plt.figure(figsize=(8, 6))
plt.stem(np.linspace(0,1,80.),y,'-')
plt.ylabel("Amplitude")
plt.title("sampled signal with sampling interval $1/80$")
plt.show()

plt.figure(figsize=(8, 6))
y = np.sin(2*np.pi*f*np.linspace(0,1,50.))
plt.stem(np.linspace(0,1,50.),y,'-')
plt.ylabel("Amplitude")
plt.title("sampled signal with sampling interval $1/50$")
plt.show()

plt.figure(figsize=(8, 6))
y = np.sin(2*np.pi*f*np.linspace(0,1,25.))
plt.stem(np.linspace(0,1,25.),y,'-')
plt.ylabel("Amplitude")
plt.title("sampled signal with sampling interval $1/20$")
plt.show()

There exists empty 'gaps' between samples. These empty gaps are points where the physical signal was not sampled. Keep this in mind that these empty gaps represent all the informations lost during the sampling process. Also note, in the case where the sampling interval is $\Delta x=\frac{1}{20}$, we can no longer see all the peaks in this sine wave.

In general, to preserve all the physical signal peaks, it is necessary to sample at least twice the maximum frequency present in the physical signal. In our case, the maximum frequency of the sine wave is $f_{max}=100\times 4$, therefore, any sampling interval $\Delta x$ above $\frac{1}{2f_{max}}$ will not exactly reconstruct the physical signal. 

#### Example: Sampling a Gaussian Pulse Signal

Let us suppose we want to sample a pulse signal in time which is a Gaussian: $d_r (t) = e^{\frac{-(t-\mu)^2}{2\sigma^2}}$.

In [ ]:
def gaussian(x, mu, sig):
    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))

x = np.linspace(-50,50,50)
mu = 0.
sig = 10.
g = gaussian(x,mu,sig)

fig = plt.figure(figsize=(10, 6))
fig.subplots_adjust(hspace=.9)

ax1 = fig.add_subplot(311)
ax1.plot(x,g)
plt.ylim(-0.5,1.5)
#ax1.axhline(y=0., xmin=-30., xmax=50., color = 'r')
plt.title("Physical signal: continuous signal")

## delta dirac comb parameter
step = 15 # sampling step
deltacom = np.ones(step)
x1 = np.linspace(-50,50,step)
ax2 = fig.add_subplot(312)
markerline, stemlines, baseline = ax2.stem(x1, deltacom, '-')
plt.title("Delta Dirac Comb: sampling function")
plt.ylim(-0.5,1.5)

## sampled Gaussian
ax3 = fig.add_subplot(313)
markerline, stemlines, baseline = ax3.stem(x1, gaussian(x1,mu,sig), '-')
plt.ylim(-0.5,1.5)
plt.title("Sampled signal")
plt.show()

The physical signal (top ) is multiplied by the Dirac Comb function (middle) and this result to the observed signal (bottom). We can write each discrete point on the sampled signal as 

$$
\begin{alignat}{2}
d_s(t)_k &= d_r(k\Delta t)
\end{alignat}
$$

Note from the example that, $\Delta t = 15$ is the sampling interval or the sampling step. The sampled signal is the sum of each discrete point on the sample grid multiplied by a Dirac delta function. 

$$
\begin{alignat}{2}
d_s(t) &= \sum_{k=-\infty}^{+\infty}\delta(t)d_s(t)_k.
\end{alignat}
$$

According to the convolution theorem which states that the Fourier transform of the product of two functions  yields to the convolution of their respective Fourier transform the relation between the time and frequency domain sampled signals is 

$$
\begin{alignat}{2}
\mathscr{F} \{d_s \}(y)  &=\Big(\mathscr{F} \big\{s\big\}\circ \mathscr{F} \big\{d_r\big\}\Big)(y),
\end{alignat}
$$

where $\mathscr{F}^{}$ denotes the Fourier transform and $\circ$ is the convolution operation. 
The Fourier transform of a Dirac Comb of regular spacing $\Delta t$, is a Dirac Comb of regular spacing $\Delta y$

$$
\begin{alignat}{2}
\mathscr{F}\big\{s\big\} & =\Delta y\sum_{k=-\infty}^{+\infty}\delta(y-k\Delta y),
\end{alignat}
$$

where $\Delta y= 2\pi/ \Delta t$  is the sampling rate. The two previous equations

$$
\begin{alignat}{2}
\mathscr{F}^{}\big\{d_s\big\}(y) &=\Big(\Delta y \sum_{k=-\infty}^{+\infty}\delta(y-k\Delta y)\Big)\circ\Big(\mathscr{F}\{d_r\}(y)\Big)\\
				  &=\Delta y \sum_{k=-\infty}^{+\infty}\mathscr{F}\{d_r\}(y-k\Delta y).
\end{alignat}
$$

For completeness, the spectrum from the point $k=0$ in the previous equation can be separated

\begin{alignat}{2}
\mathscr{F}^{}\big\{d_s\big\}(y) &= \Delta y\mathscr{F}^{}\big\{d_r\big\}(y) + \Delta y\sum_{k=-\infty, k\neq 0}^{+\infty}\mathscr{F}^{}\big\{d_r\big\}(y-k\Delta y)\\
				&\simeq \mathscr{F}^{}\big\{d_r\big\}(y) + \sum_{k=-\infty, k\neq0}^{+\infty}\mathscr{F}^{}\big\{d_r\big\}(y-k\Delta y)
\end{alignat}

Thus we see that:

1. The Fourier transform of the sampled signal duplicates the Fourier transform of the physical signal at each point in the Fourier transformed of the Dirac Comb.  
2. Sampling and the Fourier transform conserves the physical signal $\mathscr{F}^{}\big\{d_r\big\}(y)$ at the point $k=0$  and replicates it at all points $k\neq0$ with regular spacing $\Delta y$.
3. The copies for which $k\neq0$ i.e, $\sum_{k=-\infty, k\neq0}^{+\infty}\mathscr{F}^{}\big\{d_r\big\}(y-k\Delta y)$ are aliased spectra.
 
These remarks are illustrated on the following:

In [ ]:
fft_gau = np.fft.fftshift(np.abs(np.fft.fft(g)))
fft_gau = fft_gau/fft_gau.max()

fig = plt.figure(figsize=(10, 6))
fig.subplots_adjust(hspace=.9)

ax1 = fig.add_subplot(311)
ax1.plot(x,fft_gau)
plt.ylim(-0.5,fft_gau.max())
plt.title("Fourrier transform of the Physical signal: Gaussian")

## delta dirac comb parameter
nb_sample = 5
deltacom = np.ones(nb_sample)
x2 = np.linspace(-50,50,nb_sample)
ax2 = fig.add_subplot(312)
markerline, stemlines, baseline = ax2.stem(x2, deltacom, '-')
plt.title("Delta Dirac Comb: Fourier transform of the sampling function")
plt.ylim(-0.5,1.5)

## sampled Gaussian
ax3 = fig.add_subplot(313)
ax3.plot(x-0,fft_gau,'b')
ax3.plot(x-25.,fft_gau, 'k')
ax3.plot(x+25,fft_gau, 'k')
ax3.plot(x-50.,fft_gau, 'k')
ax3.plot(x+50,fft_gau, 'k')
plt.ylim(-0.5,fft_gau.max())
plt.title("Fourrier tranform of the Sampled signal")
plt.show()

The plots show that the Fourier tranform of the observed signal is a superposition of the Fourier tranform of the physical signal at each point in the Dirac Comb. At the point $k=0$ (the blue curve) is the Fourier transform of the physical signal, and all other copies at point $k\neq 0$ are aliasing copies. Which shows that any reconstruction of $d_r$ from $d_s$ requires the suppression of the aliased spectra i.e all copies of $\mathscr{F}^{}\big\{d_r\big\}$ for which $k\neq 0$.

A method to suppress the aliased copies is to multiply $\mathscr{F}^{}\big\{d_s\big\}$, by a boxcar function (this acts as a window function), as a boxcar function is unity in the pass-band (area of which $k=0$) and zero in the stop-band (area of which $k\neq 0$).

Note, a complete reconstruction of $d_r$ requires the absence of overlapping copies of the aliased spectra with the Fourier transform of the physical signal i.e $\mathscr{F}^{}\big\{d_r\big\}$ for which $k=0$. Otherwise, $d_r$ cannot be uniquely reconstructed. The following conditions must hold for a unique recovery:

1. The observed signal must be band-limited in order to avoid that $\mathscr{F}^{}\big\{d_r\big\}(y)$ at point $k=0$ extends infinitely and overlaps with its aliasing copies.
2. The sampling frequency $\Delta y$ must be greater than twice the highest frequency $f_s$ contained in the signal ($\Delta y\geq 2 f_s$). The quantity $2 f_s$ is the minimum sampling frequency or the  Nyquist rate  required to avoids aliasing.

We demonstrate this in the following example:

In [ ]:
## sampled Gaussian
plt.figure(figsize=(8, 6))
plt.plot(x-0,fft_gau,'b')
plt.plot(x-10.,fft_gau, 'k')
plt.plot(x+10,fft_gau, 'k')
plt.ylim(-0.1,fft_gau.max())
plt.title("Fourrier tranform of the Sampled signal: aliasing")
plt.show()

Let us now supose that these conditions are satisfied and that  $\sqcap^{}$ is a boxcar window, given by

$$
\begin{equation}
\sqcap^{}(y) = \bigg \{ \begin{array}{cl}
1, &  k=0 \\
0, & \mathrm{otherwise}
\end{array}
\end{equation}
$$

Let us multiply the observed signal $d_o$ by $\sqcap^{}$

$$
\begin{alignat}{2}
\mathscr{F}\big\{d_o\big\}(y) &=\sqcap^{}(y)\mathscr{F}^{}\big\{d_s\big\}(y),~~~~ Eq.8
\end{alignat}
$$

which leads to

$$
\begin{alignat}{2}
d_o(x) &= \Bigg(\mathscr{F}^{-1}\{ \sqcap^{}\}\circ d_s\Bigg)(x)~~~~ Eq.9
\end{alignat}
$$

In [ ]:
fft_gau = np.fft.fftshift(np.abs(np.fft.fft(g)))
fft_gau = fft_gau/fft_gau.max()

fig = plt.figure(figsize=(10, 6))
fig.subplots_adjust(hspace=.9)

## sampled Gaussian
ax3 = fig.add_subplot(311)
ax3.plot(x-0,fft_gau,'b')
ax3.plot(x-25.,fft_gau, 'k')
ax3.plot(x+25,fft_gau, 'k')
ax3.plot(x-50.,fft_gau, 'k')
ax3.plot(x+50,fft_gau, 'k')
plt.ylim(-0.5,fft_gau.max())
plt.title("Fourrier tranform of the Sampled signal")

ax1 = fig.add_subplot(312)
boxcar = np.zeros(len(x))
boxcar[len(x)/2-2:len(x)/2+2]=1.
ax1.plot(x,boxcar)
plt.ylim(-0.5,1.1)
plt.title("Fourrier transform of the Physical signal: Gaussian")

## delta dirac comb parameter
ax3 = fig.add_subplot(313)
ax3.plot(x-0,fft_gau,'b')
plt.ylim(-0.5,fft_gau.max())
plt.title("Fourrier tranform of the Sampled signal")

plt.show()

Since there is no overlapping aliasing spectrum with $\mathscr{F}^{}\big\{d_r\big\}(y)$ at the point $k=0$, we can write the Fourier transform of the observed signal to be

$$
\begin{alignat}{2}
\mathscr{F}\big\{d_o\big\}(y)&=\sqcap^{[k]}(y)\mathscr{F}^{}\big\{d_s\big\}(y)\\
	      &\simeq \sqcap^{[k]}(y)\Bigg(\mathscr{F}^{}\big\{d_r\big\}(y) + \sum_{k=-\infty, k\neq0}^{+\infty}\mathscr{F}^{}\big\{d_r\big\}(y-k\Delta y)\Bigg)\\
	    &\simeq\sqcap^{[k]}(y)\mathscr{F}^{}\big\{d_r\big\}(y).~~~~ Eq.10
\end{alignat}
$$

Note from the subtitution that $\sqcap^{[k]}(y)\sum_{k=-\infty, k\neq0}^{+\infty}\mathscr{F}^{}\big\{d_r\big\}(y-k\Delta y)=0$. We can rewrite Eq.10 as a convolution

$$
\begin{alignat}{2}
d_o(t) &\simeq\Bigg(\mathscr{F}^{-1}\{ \sqcap^{[k]}\}\circ d_r\Bigg)(t).~~~~ Eq.11
\end{alignat}
$$

Thus,

$$
\begin{alignat}{2}
d_s&\simeq d_r.
\end{alignat}
$$

The physical signal can be reconstructed from the observed signal if all sampling conditions are satisfied. However, ideal sampling can not be applicable in real life problems. This is due to the convolution of the physical signal with a finitie window which is the sinc filter ($\mathscr{F}^{-1}\{ \sqcap^{}\}$) and thereby could only be represented in finite support. This can be improved by increasing the window size, i.e. sampling for a longer time period.

***

* Next: [2.10 Linear Algrebra](2_10_linear_algebra.ipynb)